# 参考文献

- Deep Learning for Matching in Search and Recommendation
- https://zhuanlan.zhihu.com/p/101136699 推荐系统中的深度匹配模型
- https://zhuanlan.zhihu.com/p/267263561  推荐系统总结之深度召回模型（上）

![推荐和搜索的本质，都是match的过程](https://pic3.zhimg.com/80/v2-18e28b76d97f4421ca943d38270e801e_720w.jpg)

推荐系统和搜索应该是机器学习乃至深度学习在工业界落地应用最多也最容易变现的场景。而无论是搜索还是推荐，本质其实都是匹配，搜索的本质是给定query，匹配doc；推荐的本质是给定user，推荐item。本文主要讲推荐系统里的匹配问题，包括传统匹配模型和深度学习模型。

匹配（matching）是衡量用户对物品的兴趣的过程，也是推荐召回中的工作内容。机器学习中是以learning to match的方式根据输入表示和标记数据学习一个匹配函数。而深度学习在其发展过程中以强大的表示学习和泛化能力加上算力提升、数据规模暴涨都使得深度模型在推荐召回中大放异彩。


# 0 推荐系统概述

## 0.1 推荐系统的本质

定义：系统根据**用户的属性**（如性别、年龄、学历、地域、职业），用户在系统里**过去的行为**（例如浏览、点击、搜索、购买、收藏等），以及**当前上下文环境**（如网络、手机设备、时间等），从而给用户**推荐用户可能感兴趣的物品**（如电商的商品、feeds推荐的新闻、应用商店推荐的app等），从这个过程来看，推荐系统就是一个**给user匹配(match)感兴趣的item**的过程。

## 0.2 推荐和搜索的比较

都是一个match过程，图1.1展示的是一个搜索引擎的架构，需要match的是query和相关的doc；图1.2展示的是一个推荐系统的架构，需要match的是user(可能会带有主动意图的query)和相关的item。

![图1搜素引擎架构](https://pic1.zhimg.com/80/v2-0be4057ee616ff2acd10c92c82336694_720w.jpg)

![图2推荐系统架构](https://pic2.zhimg.com/80/v2-4f04a7013c8af1a71e285a994599b201_720w.jpg)



# 目录


- [1 feature-based的深度模型](#1-feature-based的深度模型)
    - [1.1 FM](#11-FM)

    

# 1 无序行为的表示学习

## 1.1 FM

![图1.1 FM模型的稀疏one-hot特征输入](https://pic1.zhimg.com/80/v2-5baaefa316db3345788b0256a4561134_720w.jpg)

图1.1 FM模型的稀疏one-hot特征输入

![图1.2 FM模型的数学表达](https://pic3.zhimg.com/80/v2-04cca7447d2205adb3edb26a7e1b980a_720w.jpg)

图1.2 FM模型的数学表达

对于每个输入特征，模型都需要学习一个低维的隐向量表示$v$，也就是NN中的embedding层。FM由两部分组成：蓝色的是LR线性模型，红色的是二阶特征组合——两两特征组合，但不和自己组合，向量之间的交叉用向量内积表示。

```python
feats = features['feats']  # [batch_size*feat_num, feat_val_num]
emb = self.emb_module(feats)  # (bs*feature_num, feat_val_num, emb_dim) feat_val_num=1
emb = tf.reshape(emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num, emb_dim) feat_val_num=1 

# -----LR-----
linear_emb = self.linear_emb_module(feats)  # (bs*feature_num, feat_val_num, emb_size) feat_val_num=1,emb_size=1
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1]) # （bs,feat_num）
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True)  # [bs, 1]

# -----二阶-----
# The shape of embedding is [bs, feature_num, emb_dim]
summed_features_emb_square = tf.square(tf.reduce_sum(emb,1))  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
squared_sum_features_emb = tf.reduce_sum(tf.square(emb),1)  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
FM2 = 0.5 * tf.subtract(summed_features_emb_square, squared_sum_features_emb)  # [bs, emb_dim] 
# [bs, emb_dim]的FM结果与特征两两逐元素相乘加和再tf.reduce_sum(pairwise_interaction, 1)消除第二维度（feat_num*(feat_num-1)/2）得到的[bs, emb_dim]结果一致

FM = tf.reduce_sum(FM2, 1) + linear_out # [bs,] 按照FM计算公式有此步骤

```

## 1.2 Wide&Deep

-  [2016]"Wide & Deep Learning for Recommender Systems": https://arxiv.org/abs/1606.07792 [2] [official] https://github.com/tensorflow/models/tree/master/official/wide_deep 


![wd模型图](https://github.com/hsux/Distributed-training/raw/master/images/wd_imgs/wd.jpg)

（1） 记忆性：wide部分长处在于学习样本中的**高频**部分。
- 优点是模型的**记忆性好**，对于样本中出现过的**高频低阶特征能够用少量参数学习**；
- 缺点是模型的**泛化能力差**，例如对于没有见过的ID类特征，模型学习能力较差。

（2） 泛化性：deep部分长处在于学习样本中的**长尾**部分。
- 优点是**泛化能力强**，对于**少量出现过的样本甚至没有出现过的样本都能做出预测（非零的embedding向量）**，容易带来惊喜。
- 缺点是模型**对于低阶特征的学习需要用较多参数才能等同wide部分效果**，而且泛化能力强某种程度上也**可能导致过拟合出现bad case**。尤其对于冷启动的一些item，也有可能给用户带来惊吓。

![模型框架2](https://pic1.zhimg.com/80/v2-0bb7650022db2d677b029bb39537ed60_720w.jpg)

虽然模型的deep部分拟合和泛化能力很强，但绝对不意味着把特征交叉都交给MLP就够了。实际证明，对于重要的一些人工经验的特征，对于提升整体效果还是非常重要的，如上图所示。这个人工特征的所谓缺点，也是后续各种模型结构想对其进行“自动化”的优化点。


## 1.3 Deep Crossing

- 2016 微软

wide&deep的MLP部分是全连接网络，每层的输入是前一层的输出，受限于模型结构，**越往后的层越难学习到原始输入的表达，一般深度不会太深**，超过5层的网络在工业界已经算很少见了。为了解决这个问题，deep crossing网络引入了resnet残差网络的概念，**通过`short-cut`，在MLP的深层网络，也能接收来自第一层的输入，这样可以使得模型的深度达到10层之多**。

![deep crossing模型框架](https://pic4.zhimg.com/80/v2-cc6b972eb9e73816a454d154413a4a73_720w.jpg)

上述提到的wide&deep以及deep crossing框架更像是在模型结构做的改进，一个引入了wide&deep，一个引入了resnet，**特征层面并没有做太多改造**，如何体现feature-base呢？sigIR2017就有一篇文章做了个实验，对wide&deep以及Deep&Cross实验*按照embedding是否做初始化分别做了实验*。实验发现，如果embedding是随机初始化的，两个深度模型连基础的FM模型都打不过；哪怕经过FM初始化了embedding，wide&deep模型效果也仅仅略好于FM模型，而deep crossing模型依然比不过FM模型，实验结果如下图所示：

![不同初始化对模型影响](https://pic3.zhimg.com/80/v2-07bbaf8ac3eafe8a87ba09b50f39db26_720w.jpg)

全连接网络表面上看对所有节点都进行了连接，理论上应该学习到了各个节点的交叉特征，但是从结果上来看，**MLP对这些特征交叉的学习能力确实非常差的**。纠其原因，还是在模型结构的设计上。

![wide&deep模型和deep crossing模型](https://pic1.zhimg.com/80/v2-a76f166f96a59270b7108b352d48e974_720w.jpg)

上图中两种模型，embedding到MLP之间都是做emb-concat，concat的结果表达的交叉信息有限，仅靠MLP想完全捕捉有效交叉信息是困难的，于是在embedding与MLP之间利用更多的数学先验范式做特征交叉是一个改进的方向。

## 1.4 PNN

- 在embedding与MLP之间引入product layer显式地学习每个field的embedding向量间的两两交叉。

![PNN模型框架](https://pic2.zhimg.com/80/v2-6a31278d8d55c8516a891c088c287a79_720w.jpg)

上图中，product layer左边z维embedding层的线性部分，右边p为embedding层的特征交叉部分。思想来源于推荐系统中特征间的交叉关系更多是一种“且”的关系而非“加”的关系。例如，性别为男且喜欢游戏的人群，比起性别男和喜欢游戏的人群，前者的组合比后者更能体现特征交叉的意义。

根据product的方式不同，可以分为inner product（IPNN）和outer product(OPNN)，如下图所示：

![PNN模型的两种不同交叉方式](https://pic3.zhimg.com/80/v2-695f940fca822ff863f6393631fed7ea_720w.jpg)

其中，IPNN模型每个特征是个inner product, f个field两两交叉，得到的新的特征组合有$f*(f-1)/2$个；outer product是两个向量的乘积，得到的新的特征组合有$f*(f-1)/2*k*k$个。

```python

feats = features['feats']  # [batch_size*feat_num, feat_vals]
pairwise_emb = self.pairwise_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim) 
pairwise_emb = tf.reshape(pairwise_emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num, emb_dim)
        
# ----z part. [bs, self.total_feat_num, self.emb_dim] -> [bs, self.total_feat_num*self.emb_dim]
z_part = tf.reshape(pairwise_emb, [-1, self.total_feat_num*self.emb_dim]) 

# IPNN-------------------------
class PairwiseInteraction(BaseModule):
    '特征emb两两元素乘'
    def __init__(self, name, feat_num):
        super(PairwiseInteraction, self).__init__(name)
        self.feat_num = feat_num
        
    @tf.Module.with_name_scope
    def __call__(self, embeddings)
        ''' embeddings: [bs, feat_num, emb_dim]
            return [bs, feat_num*(feat_num-1)/2, emb_dim]
        '''
        elementwise_list = []
        for i in range(self.feat_num):
            for j in range(i+1, self.feat_num):
                elementwise_list.append(tf.multiply(embeddings[:,i,:],embeddings[:,j,:]))  # [(bs,emb_dim),....,] 共feat_num*(feat_num-1)/2个
                
        pairwise_interaction = tf.stack(elementwise_list)  # [feat_num * (feat_num - 1) / 2, bs, emb_dim]
        pairwise_interaction = tf.transpose(pairwise_interaction, perm=[1,0,2])  # [bs, feat_num * (feat_num - 1) / 2, emb_dim]
        return pairwise_interaction  # 交互项矩阵
    
self.p_part_module = PairwiseInteraction(
    name='p_part_module',
    feat_num=self.total_feat_num,
)
p_part = self.p_part_module(pairwise_emb)
p_part = tf.reshape(tf.reduce_sum(p_part, axis=-1), [-1, self.total_pairs])

# OPNN--------------------
if outer_mode == 'mat':
    self.outer_kernel = OuterKernel(
        name='outer_kernel',
        shape=[self.emb_dim,self.total_pairs,self.emb_dim]
    )
elif outer_mode == 'vec':
    self.outer_kernel = OuterKernel(
        name='outer_kernel',
        shape=[self.total_pairs,self.emb_dim]
    )
else: # outer_mode == 'num'
    self.outer_kernel = OuterKernel(
        name='outer_kernel',
        shape=[self.total_pairs,1]
    ) 

if self.product_mode == 'outer':
    p, q = self.pairs_emd_module(pairwise_emb)  # [bs,total_pairs,emb_dim]
    if self.outer_mode == 'mat':
        # p与outer_kernel元素积，结果再与q元素积
        p = tf.expand_dims(p,1)  # [bs, total_pairs, emb_dim] -> [bs,1,total_pairs,emb_dim]
        p_part = tf.reduce_sum(  # [bs, total_pairs]
                 tf.multiply(  # [bs, total_pairs, emb_dim]
                     tf.transpose(  # [bs, total_pairs, emb_dim]
                         tf.reduce_sum(  # [bs, emb_dim, total_pairs]
                             tf.multiply(p, self.outer_kernel),  # [bs,emb_dim,total_pairs,emb_dim]
                             axis=-1
                         ),  # reduce_sum
                         perm=[0,2,1]
                     ),  # transpose
                     q
                 ),  # multiply
                 axis=-1
             )  # reduce_sum
    else:
        # p,q,kernel元素积
        kernel = tf.expand_dims(self.outer_kernel,0)  # vec:[1,total_pairs,emb_dim],num:[1,total_pairs,1] 
        p_part = tf.reduce_sum(p*q*kernel, axis=-1)  # [bs,total_pairs]
# ----deep net
# [bs, self.total_feat_num*self.emb_dim] concat [bs, feat_num*(feat_num-1)/2] -> [bs, self.total_emb_dim+feat_num*(feat_num-1)/2]
deep_input = tf.concat([z_part, p_part], axis=1)
```

## 1.5 DeepFM

- [IJCAI 2017]"DeepFM: A Factorization-Machine based Neural Network for CTR Prediction": https://arxiv.org/abs/1703.04247v1


wide&deep框架虽然强大，但是LR模型需要人工进行特征工程，华为诺亚方舟团队结合FM相比LR的特征交叉，在17年提出了DeepFM，将wide&deep中的LR部分替换为FM，避免人工特征工程，如下图所示：

![deepfm框架](https://pic3.zhimg.com/80/v2-aef79867ec40a8327366333d5764ede2_720w.jpg)

- 更强的低阶特征表达：wide部分取代WDL的LR，与wide&deep模型以及deep crossing模型相比更能捕捉低阶特征信息

- embedding层共享：wide&deep部分的embedding层需要针对deep部分单独设计；而在deepFM中，FM和DEEP部分共享embedding层，FM训练得到的参数既作为wide部分的输出，也作为DNN部分的输入。

- end-end训练：embedding和网络权重联合训练，无需预训练和单独训练

- 缺点：FM部分输出emb_size可能比dnn输出部分小太多

```python
feats = features['feats']  # [batch_size*feat_num, feat_vals]
linear_emb = self.linear_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, 1) 
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1])
second_emb = self.pairwise_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim) 
second_emb = tf.reshape(second_emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num * feat_val_num * emb_dim) feat_val_num=1 in dense

# ----Linear Module
# [bs, feat_num] -> [bs, 1]
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True) 

# ----FM Module
# [bs, feat_num, emb_dim] -> [bs, emb_dim]
# The shape of embedding is [bs, feature_num, emb_dim]
summed_features_emb_square = tf.square(tf.reduce_sum(second_emb,1))  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
squared_sum_features_emb = tf.reduce_sum(tf.square(second_emb),1)  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
FM = 0.5 * tf.subtract(summed_features_emb_square, squared_sum_features_emb)  # [bs, emb_dim] 

# ----Deep Module
# [bs, total_emb_dim] -> [bs, dnn_layers[-1]]
dnn_out = self.dnn_module(tf.reshape(second_emb,shape=[-1, self.total_emb_dim]), is_train)

# ----second order out
# [bs, emb_dim + dnn_layers[-1]] -> [bs, 1]
second_input = tf.concat([FM_out, dnn_out], axis=1)
second_order_out = self.second_order_out(second_input, is_train)  #dense * 1

# ----DeepFM output
global_bias = tf.Variable(name='global_bias', initial_value=tf.zeros_initializer()(shape=[1]), dtype=tf.float32)
global_bias = global_bias * tf.ones_like(global_bias, dtype=tf.float32)  # [bs, 1]  否则报错
model_out = tf.add_n([global_bias, linear_out, second_order_out])  # [bs, 1]
model_out = self.out_act(model_out)
```

## 1.6 NFM(Neural Factorization Machines)

- [SIGIR 2017]"Neural Factorization Machines for Sparse Predictive Analytics": https://arxiv.org/abs/1708.05027
- https://github.com/hexiangnan/neural_factorization_machine

deepFM在embedding层后把FM部分直接**concat**起来（f\*k维，f个field，每个filed是k维向量）作为DNN的输入。Neural Factorization Machines，简称NFM，提出了一种更加简单粗暴的方法，在embedding层后，做了一个叫Bi-interaction的操作，让**各个field做element-wise后sum**起来去做特征交叉，MLP的**输入规模直接压缩到k维**，和特征的原始维度ｎ和特征field维度f没有任何关系，如下图所示：

![NFM模型框架](https://pic1.zhimg.com/80/v2-f38d5cee16581c7af469ba13af7f82a4_720w.jpg)

图中只画出了其中的deep部分, wide部分在这里省略没有画出来。Bi-interaction所做的操作很简单：让**f个field两两element-wise相乘**后，得到f*(f-1)/2个维度为k的向量，然后直接**sum**起来，最后得到一个k维的向量。所以该层**没有任何参数需要学习，同时也降低了网络复杂度，能够加速网络的训练**；但同时这种方法也**可能带来较大的信息损失**。

- 就是把DeepFM中的FM和dnn并联改成了FM和dnn串联

```python
feats = features['feats']  # [batch_size*feat_num, feat_val_num]
linear_emb = self.linear_emb_module(feats)  # dense:(bs*feature_num, feat_val_num, 1) feat_val_num=1 in dense; sparse:(bs*feature_num, 1) because combiner
linear_emb = tf.reshape(linear_emb, [-1, self.total_feat_num*1])
emb = self.emb_module(feats)  # dense:(bs*feature_num, feat_val_num, emb_dim) feat_val_num=1 in dense; sparse:(bs*feature_num, emb_dize) because combiner
emb = tf.reshape(emb, [-1, self.total_feat_num, self.emb_dim])  # (bs, feat_num, emb_dim) feat_val_num=1 in dense; 

# Model
# 原文中此处是x_i为one-hot编码的0或1，样本Xone-hot向量的第i个特征，求加权和
# 官方实现如下
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True)  # [bs, 1]

# ---- Bi_Interaction
# emb shape is [bs,feature_num,emb_dim]
# The shape of embedding is [bs, feature_num, emb_dim]
summed_features_emb_square = tf.square(tf.reduce_sum(second_emb,1))  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
squared_sum_features_emb = tf.reduce_sum(tf.square(second_emb),1)  # [bs,feature_num,emb_dim] -> [bs, emb_dim]
bi_out = 0.5 * tf.subtract(summed_features_emb_square, squared_sum_features_emb)  # [bs, emb_dim] 

# -----Deep
deep_out = self.deep_module(bi_out, is_train=is_train)  # [bs, deep_layer[-1]]
deep_out = self.final_module(deep_out, is_train=is_train)  # [bs, 1]

# NFM output
global_bias = tf.Variable(name='global_bias', initial_value=tf.zeros_initializer(), shape=[1]), dtype=tf.float32)
global_bias = global_bias * tf.ones_like(global_bias, dtype=tf.float32)  # [bs, 1]  否则报错
model_out = tf.add_n([global_bias, linear_out, deep_out])  # [bs, 1] must equal rank
model_out = self.out_act(model_out)
```

## 1.7 AFM

提到的各种网络结构中的FM在做特征交叉时，让不同特征的向量直接做交叉，基于的假设是**各个特征交叉对结果的贡献度是一样的**。这种假设往往不太合理，原因是**不同特征对最终结果的贡献程度一般是不一样的**。Attention Neural Factorization Machines，简称AFM模型，利用了近年来在图像、NLP、语音等领域大获成功的attention机制，在前面讲到的NFM基础上，引入了attention机制来解决这个问题，如下图所示。

![AFM](https://pic1.zhimg.com/80/v2-eb07df13a0010c275bbf8ecba72edb2c_720w.jpg)

AFM的embedding层后和NFM一样，先让f个field的特征做了element-wise product后，得到f*(f-1)/2个交叉向量。和NFM直接把这些交叉项sum起来不同，AFM引入了一个Attention Net，认为这些交叉特征项每个对结果的贡献是不同的，例如xi和xj的权重重要度，用aij来表示。从这个角度来看，其实AFM其实就是个**加权累加**的过程。Attention Net部分的权重aij不是直接学习，而是通过如下公式表示:

![AFM公式](https://pic1.zhimg.com/80/v2-941f8604f6e7644f2648410be6da9094_720w.jpg)
![](https://pic4.zhimg.com/80/v2-9bd6536205efe338fc4b1deb182d96c3_720w.png)

其中t表示attention net中的隐层维度，k和前面的一样，为embedding层的维度emb_size。需要学习的参数只有3个，$W,b,h$，参数个数共有$t*k+2*t$个。得到aij权重后，对各个特征两两点积加权累加后，得到一个k维向量，引入一个简单的参数向量pT,维度为k进行学习，和wide部分一起得到最后的AFM输出。

![AFM模型中attention的可视化解释](https://pic3.zhimg.com/80/v2-4b3a096d531539fa495495f2e7d2d7aa_720w.jpg)

关于AFM还有个好处，通过attention-base pooling计算的score值aij体现的是特征vi和vj之间的权重，能够选择有用的二阶特征，如上图所示。

```python
# ----Linear Module
# 官方reduce_sum, [bs, self.total_feat_num] -> [bs, 1]
linear_out = tf.reduce_sum(linear_emb, 1, keep_dims=True) 

# ----Pairwise Interaction Module
# feat vec两两逐元素相乘 [bs, self.total_feat_num, emb_dim] -> [bs, feat_num * (feat_num - 1) / 2, emb_dim]
pairwise_interaction_out = self.pairwise_interaction_module(pairwise_emb) 

# ----Attention Module 公式()第三部分，官方没有h
if self.use_att:
    # relu(wx+b): [bs, feat_num * (feat_num - 1) / 2, emb_dim] -> [bs, feat_num * (feat_num - 1) / 2, att_layer[-1]]
    attention_wx_plus_b = self.attention_nn_module(pairwise_interaction_out, is_train) 
    # P*attention_wx_plus_b: [bs, feat_num * (feat_num - 1) / 2, att_layer[-1]] -> [bs, feat_num * (feat_num - 1) / 2, 1]
    attention_p_mul = tf.reduce_sum(tf.multiply(self.attention_p, attention_wx_plus_b)), 2, keep_dims=True)  
    # 官方代码增加dropout
    attention_score = tf.nn.dropout(tf.softmax(self.attention_p_mul), self.att_keep_prob)  # [bs, feat_num * (feat_num - 1) / 2, 1]

# ----Attention-aware Pairwise Interaction Layer
if self.use_att:
    # 在FM计算reduce_sum之前对两两特征交叉的结果进行attention
    # att_score * PI: [bs, feat_num * (feat_num - 1) / 2, 1] * [bs, feat_num * (feat_num - 1) / 2, emb_dim] -> [bs, emb_dim]
    afm = tf.reduce_sum(tf.multiply(attention_score, pairwise_interaction_out), axis=1, name='afm')   # [bs, emb_dim]
else:
    # [bs, feat_num * (feat_num - 1) / 2, emb_dim] -> [bs, emb_dim]
    afm = tf.reduce_sum(pairwise_interaction_out, axis=1, name='afm')   # [bs, emb_dim]
afm = tf.nn.dropout(afm, self.afm_keep_prob)  # 官方
# second_order output
# w matmul afm: [bs, emb_dim] matmul [emb_dim, 1] -> [bs, 1]
second_order_out = self.second_order_out(afm, is_train)

# AFM output
global_bias = tf.Variable(name='global_bias', initial_value=tf.zeros_initializer()(shape=[1]), dtype=tf.float32)
global_bias = global_bias * tf.ones_like(global_bias, dtype=tf.float32)  # [bs, 1]  否则报错 
model_out = tf.add_n([global_bias, linear_out, second_order_out], name='afm_out')  # [bs, 1]  must equal rank
model_out = self.out_act(model_out)
```